# OPTIMIZATION ALGORITHMS that we can apply to e.g. obtain more of compound X

- OptKnock: determines which are the reactions we have to knock out in order to optimize production of X
- OptStrain: determines which genes we have to insert to our model so that it produces more of X (but it was already being produced) OR start producing X from scratch 
- OptGene: determines, within a whole population (so taking into account evolutionary driving forces), which deletions should be made to improve production of X 
- OptReg: tells us which knock-outs, upregulatins or downregulations we have to make to improve production of X
- OptForce: tells us which (more than 1 modification) knock-outs, upregulatins or downregulations we have to make to improve production of X to a specific flux

In [2]:
from cobra.io import read_sbml_model
model = read_sbml_model('iCre1355_hetero_V2.xml')
model.metabolites.get_by_id('6mpur_c')

Metabolite identifier,6mpur_c
Name,6-Mercaptopurine
Memory address,0x07f792c8873c8
Formula,C5H4N4S
Compartment,c
In 2 reaction(s),"6MPURPRT, 6MPURt"


In [ ]:
hetero = read_sbml_model('iCre1355_hetero.xml')
auto = read_sbml_model('iCre1355_auto.xml')
mixo = read_sbml_model('iCre1355_mixo.xml')

write_sbml_model(hetero, 'iCre1355_hetero_V2.xml')
write_sbml_model(auto, 'iCre1355_auto_V2.xml')
write_sbml_model(mixo, 'iCre1355_mixo_V2.xml')

hetero.name = 'iCre1355_Hetero'
auto.name = 'iCre1355_Auto'
mixo.name = 'iCre1355_Mixo'

In [3]:
model.reactions.get_by_id('6MPURPRT')

Reaction identifier,6MPURPRT
Name,6-mercaptopurine phosphoribosyltransferase
Memory address,0x07f792c126a20
Stoichiometry,6mpur_c + prpp_c --> 6tins5mp_c + ppi_c 6-Mercaptopurine + 5-Phospho-alpha-D-ribose 1-diphosphate --> 6-Thioinosine-5'-monophosphate + Diphosphate
GPR,Cre17.g708100.t1.2 or Cre04.g217934.t1.1 or Cre08.g358900.t1.2
Lower bound,0.0
Upper bound,1000.0


In [4]:
model.reactions.get_by_id('6MPURt')

Reaction identifier,6MPURt
Name,"nucleobase transport, 6-mercaptopurine (extracellular)"
Memory address,0x07f792b580550
Stoichiometry,6mpur_e <=> 6mpur_c 6-Mercaptopurine <=> 6-Mercaptopurine
GPR,Cre10.g433200.t1.2 or Cre10.g442600.t1.2 or Cre10.g442600.t2.1 or Cre10.g442800.t1.1 or Cre06.g30...
Lower bound,-1000.0
Upper bound,1000.0


In [6]:
model.metabolites.query('palmit', 'name')

[<Metabolite 3hpalmACP_h at 0x7f792c8dac18>,
 <Metabolite retpalm_s at 0x7f792c70ac18>,
 <Metabolite retpalm_11_cis_s at 0x7f792c70acf8>]

In [8]:
from cobra import Reaction, Metabolite
from cameo import models

In [15]:
name = []
ID = []

for metabolite in model.metabolites.query('H2', 'name'):
    
    print(metabolite, '\t', metabolite.name)    
    if metabolite.name == 'H2':
        name.append(metabolite.name)
        ID.append(metabolite.id)
        
print(name, ID)

h2_c 	 H2
h2_e 	 H2
h2_h 	 H2
h2_m 	 H2
h2o_c 	 H2O
h2o_e 	 H2O
h2o_f 	 H2O
h2o_h 	 H2O
h2o_m 	 H2O
h2o_n 	 H2O
h2o_s 	 H2O
h2o_u 	 H2O
h2o_x 	 H2O
['H2', 'H2', 'H2', 'H2'] ['h2_c', 'h2_e', 'h2_h', 'h2_m']


In [17]:
reactions = []

for element in ID:
    compound = model.metabolites.get_by_id(element)
    for react in compound.reactions:
        reactions.append(react.id)

for element in reactions:
    print(model.reactions.get_by_id(element))

H2th: h2_c <=> h2_h
H2ti: h2_c --> h2_e
HYDA: 2.0 fdxrd_c <=> 2.0 fdxox_c + h2_c + 2.0 h_c
H2tm: h2_c <=> h2_m
EX_h2_e: h2_e --> 
H2ti: h2_c --> h2_e
H2th: h2_c <=> h2_h
HYDAh: 2.0 fdxrd_h <=> 2.0 fdxox_h + h2_h + 2.0 h_h
HYDAm: 2.0 fdxrd_m <=> 2.0 fdxox_m + h2_m + 2.0 h_m
H2tm: h2_c <=> h2_m


The reactions that actually produce hydrogen are HYDA (in cytosol), HYDAh (in chloroplast) and HYDAm (in mitochondria); the other ones just represent transport of hydrogen between different compartments. The 3 of them are the same reaction, just in different compartments. From the cytoplasm is also extruded to outside of the organism.

In [10]:
model.compartments

{'c': 'Cytosol',
 'h': 'Chloroplast',
 'm': 'Mitochondria',
 'x': 'Glyoxysome',
 'f': 'Flagellum',
 'e': 'Extra-organism',
 'n': 'Nucleus',
 'g': 'Golgi Apparatus',
 's': 'Eyespot',
 'u': 'Thylakoid Lumen',
 'i': 'Inner Mitochondrial membrane space'}

In [23]:
model.reactions.HYDA #the enzyme is (FeFe)-hydrogenase acting in the cytoplasm

Reaction identifier,HYDA
Name,"(FeFe)-hydrogenase, cytoplasm"
Memory address,0x07f792b77e748
Stoichiometry,2.0 fdxrd_c <=> 2.0 fdxox_c + h2_c + 2.0 h_c 2.0 Reduced ferredoxin <=> 2.0 Oxidized ferredoxin + H2 + 2.0 H+
GPR,( Cre03.g199800.t1.1 or Cre09.g396600.t1.1 ) and ( Cre01.g006100.t1.2 or Cre12.g487900.t1.2 or Cr...
Lower bound,-1000.0
Upper bound,1000.0


In [21]:
model.reactions.HYDAh #the enzyme is (FeFe)-hydrogenase acting in the chloroplast

Reaction identifier,HYDAh
Name,"(FeFe)-hydrogenase, chloroplast"
Memory address,0x07f792b77efd0
Stoichiometry,2.0 fdxrd_h <=> 2.0 fdxox_h + h2_h + 2.0 h_h 2.0 Reduced ferredoxin <=> 2.0 Oxidized ferredoxin + H2 + 2.0 H+
GPR,( Cre03.g199800.t1.1 or Cre09.g396600.t1.1 ) and ( Cre16.g658400.t1.2 or Cre17.g700950.t1.2 or Cr...
Lower bound,-1000.0
Upper bound,1000.0


In [22]:
model.reactions.HYDAm #the enzyme is (FeFe)-hydrogenase acting in the mitochondria

Reaction identifier,HYDAm
Name,"(FeFe)-hydrogenase, mitochondria"
Memory address,0x07f792b794b00
Stoichiometry,2.0 fdxrd_m <=> 2.0 fdxox_m + h2_m + 2.0 h_m 2.0 Reduced ferredoxin <=> 2.0 Oxidized ferredoxin + H2 + 2.0 H+
GPR,( Cre03.g199800.t1.1 or Cre09.g396600.t1.1 ) and Cre12.g559950.t1.2
Lower bound,-1000.0
Upper bound,1000.0


In [37]:
hetero.reactions.Biomass_Chlamy_mixo

Reaction identifier,Biomass_Chlamy_mixo
Name,Biomass (mixotrophic) updated
Memory address,0x07f7925e4d908
Stoichiometry,0.0370621525868795 ac_c + 3.95421018705422e-06 acaro_h + 0.279255294881266 alatrna_c + 7.90842037410844e-07 anxan_u + 0.524141412987151 arab_L_c + 0.093656422877759 argtrna_c + 0.0695383636530289 a... 0.0370621525868795 Acetate + 3.95421018705422e-06 alpha-Carotene + 0.279255294881266 L-Alanyl-tRNA(Ala) + 7.90842037410844e-07 Antheraxanthin + 0.524141412987151 L-Arabinose + 0.093656422877759 L-A...
GPR,
Lower bound,0.0
Upper bound,0.0


In [41]:
mixo.objective = model.reactions.Biomass_Chlamy_mixo
print(model.optimize().objective_value)

0.0


In [46]:
with mixo:
    mixo.reactions.Biomass_Chlamy_mixo.upper_bound = 1000
    mixo.objective = mixo.reactions.Biomass_Chlamy_mixo
    print(mixo.optimize().objective_value)

0.15158877721813943


In [44]:
mixo.medium

{'EX_h_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_nh4_e': 1.0,
 'EX_so4_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_na1_e': 10.0,
 'EX_photonVis_e': 80.0,
 'EX_o2_e': 10.0,
 'EX_co2_e': 2.0,
 'EX_ac_e': 2.0}

In [47]:
medium = mixo.medium
with mixo:
    mixo.reactions.Biomass_Chlamy_mixo.upper_bound = 1000
    mixo.objective = mixo.reactions.Biomass_Chlamy_mixo
    medium['EX_so4_e'] = 0
    model.medium = medium
    print(mixo.optimize().objective_value)
    print(mixo.optimize().fluxes['Biomass_Chlamy_mixo']) #this gives us the growth rate/biomass production...
    print(medium)

0.15158877721813943
0.15158877721813943
{'EX_h_e': 10.0, 'EX_h2o_e': 10.0, 'EX_pi_e': 10.0, 'EX_nh4_e': 1.0, 'EX_so4_e': 0, 'EX_fe2_e': 10.0, 'EX_mg2_e': 10.0, 'EX_na1_e': 10.0, 'EX_photonVis_e': 80.0, 'EX_o2_e': 10.0, 'EX_co2_e': 2.0, 'EX_ac_e': 2.0}
